In [1]:
import os
import sys
import argparse
import argparse
import netCDF4 as nc
import numpy as np

In [2]:
ifile= 'C:\\Users\\Utilisateur\\Desktop\\internship\\data\\input\\20180404T205505_S1B_AMSR2_Icechart-Greenland-CentralWest.nc'
ds = nc.Dataset(ifile, format="NETCDF4")
p_codes = ds['polygon_codes'][:]
print(p_codes)

['id;CT;CA;SA;FA;CB;SB;FB;CC;SC;FC;CN;CD;CF;POLY_TYPE'
 '41;92;-9;87;8;-9;-9;-9;-9;-9;-9;98;-9;-9;I'
 '42;92;30;91;8;70;87;-9;-9;-9;-9;-9;-9;-9;I'
 '43;92;10;91;8;60;87;-9;30;83;-9;98;-9;-9;I'
 '44;50;20;93;5;20;91;5;10;87;4;98;-9;-9;I'
 '45;92;30;91;8;70;87;-9;-9;-9;-9;-9;-9;-9;I'
 '46;80;30;91;5;30;87;4;20;87;3;-9;-9;-9;I'
 '47;80;20;93;6;20;91;5;40;87;4;-9;-9;-9;I'
 '48;92;-9;91;8;-9;-9;-9;-9;-9;-9;98;-9;-9;I'
 '49;92;80;91;8;20;87;-9;-9;-9;-9;98;-9;-9;I'
 '50;30;10;87;3;10;83;2;10;82;2;-9;-9;-9;I'
 '51;91;40;93;6;50;91;6;10;87;5;-9;-9;-9;I'
 '52;91;60;93;7;40;91;6;-9;-9;-9;-9;-9;-9;I'
 '53;92;-9;91;8;-9;-9;-9;-9;-9;-9;98;-9;-9;I']


In [3]:
def ice_stage(stage):
    """
    Gives back the index the concentration or 1 should be on.
    Each index corresponds to a particular ice type
    (0: Young ice; 1: First Year ice; 2: Multi year ice ; 3: Ice free).
    The values on which depends this classification are described ine the
    ASIP-v2 manual.
    Parameters
    ----------
    stage : integer
        stage of development.
    Returns
    -------
    index_ : integer
        index of the list where the value (0/1 or concentration) will be.
    """
    index_= None

    if stage ==0:
        #print('ice_free')
        index_ = 0
    if stage in range(81,86):
        #print('Young ice')
        index_=1
    if stage in range(86,94):
        #print('First year ice')
        index_=2
    if stage in range(95,98):
        #print('multiyear ice')
        index_=3
    return index_

In [4]:
def ice_form(form):
    """
    
    """
    index_= None

    if form in range(0,3):
        #print('Pancake Ice')
        index_ = 0
    if form in range(3,5):
        #print('Small Floe')
        index_=1
    if form in range(5,8):
        #print('Big Floe')
        index_=2
    if form == 8:
        #print('fast ice')
        index_=3
    if form in range(9,11):
        #print('iceberg')
        index_=4
    if form in range(11,21):
        #print('strips and patches')
        index_=5
    return index_

In [5]:
def one_hot_binary(ct,ca,sa,fa,cb,sb,fb,cc,sc,fc, min_ct=10):

    """
    Returns the list of one-hot encoded values corresponding to ice types
    based on concentration and stage of development of thickest, second
    thickest and thrid thickest ice

    Parameters
    ----------
    ct : integer
        Total concentration.
    ca : integer
        Partial concentration of thickest ice.
    sa : integer
        Stage of development of thickest ice.
    fa : integer
        Form of thickest ice.
    cb : integer
        Partial concentration of second thickest ice.
    sb : integer
        Stage of development of second thickest ice.
    fb : integer
        Form of second thickest ice.
    cc : integer
        Partial concentration of third thickest ice.
    sc : integer
        Stage of development of third thickest ice.
    fc : integer
        Form of third thickest ice.
    Returns
    -------
    result : list
        List of one-hot encoded values corresponding to ice types and ice forms.
    """

    result = [0]*20
    if ct < min_ct:# open water
        result[0]=1
        return result 

    f = [0]*18 # fractions
    for ci, si, fi in zip ([ca, cb, cc], [sa, sb, sc], [fa, fb, fc]):
        icestage = ice_stage(si)
        iceform = ice_form(fi)
        if (ci != -9) and (icestage is not None) and (iceform is not None):
            
            index = ((icestage-1)*6)+(iceform)
            f[index] += ci
    if max(f) == 0:
        icestage = ice_type(sa)
        iceform = ice_form(fa)
        index = ((icestage-1)*6)+(iceform)+1
    else:
        index = np.argmax(f)+1
    if index is not None:
        result[index]=1
    
    
    #distance to border
    
    
    return result

In [10]:
def one_hot_continous(ct,ca,sa,fa,cb,sb,fb,cc,sc,fc):
    """

    Returns the list of one-hot encoded values in terms of concentration
    corresponding to ice types based on concentration and stage of development
    of thickest, second thickest and thrid thickest ice.

    Parameters
    ----------
    ct : integer
        Total concentration.
    ca : integer
        Partial concentration of thickest ice.
    sa : integer
        Stage of development of thickest ice.
    fa : integer
        Form of thickest ice.
    cb : integer
        Partial concentration of second thickest ice.
    sb : integer
        Stage of development of second thickest ice.
    fb : integer
        Form of second thickest ice.
    cc : integer
        Partial concentration of third thickest ice.
    sc : integer
        Stage of development of third thickest ice.
    fc : integer
        Form of third thickest ice.
    Returns
    -------
    result : list
        List of one-hot encoded (in terms of concentration) values
        corresponding to ice types and ice forms.
    """
    result = [0]*20
    for si, ci, fi in zip ([sa, sb, sc], [ca, cb, cc], [fa, fb, fc]):
        icestage = ice_stage(si)
        iceform = ice_form(fi)
        if (ci != -9) and (icestage is not None) and (iceform is not None):
            index = ((icestage-1)*6)+(iceform)+1
            result[index] += ci/100
    if max(result) == 0:
        result[0] = 1-(ct/100)
        icestage = ice_stage(sa)
        iceform = ice_form(fa)
        index = ((icestage-1)*6)+(iceform)+1
        if index is not None  :
            result[index] = ct/100
    else:
        result[0] = 1-sum(result[1:])

    return result

In [14]:
print(['W','Pancake_YI','Mediumfloe_YI','Bigfloe_YI', 'FastI_YI','Iceberg_YI, Strips_YI'])
print(one_hot_binary(50, 20, 93, 5, 20, 91, 5, 10, 87, 4))
print(one_hot_continous(50, 20, 93, 5, 20, 91, 5, 10, 87, 4))

['W', 'Pancake_YI', 'Mediumfloe_YI', 'Bigfloe_YI', 'FastI_YI', 'Iceberg_YI, Strips_YI']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0.5, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
